# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [8]:
# your code goes here

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')

df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [9]:
def limpito(datitos: pd.DataFrame):
    dataframe = datitos.copy()
    dataframe.columns = [x.replace('EmploymentStatus','Employment Status') if x == 'EmploymentStatus' else x for x in dataframe.columns ]
    # Todas las columnas en minuscula y espacio por _:
    dataframe.columns = [x.lower().replace(' ','_') for x in dataframe.columns]
    # Cambiar columna st por state
    dataframe.columns = [x.replace('st','state') if x == 'st' else x for x in dataframe.columns ]
    # Unificar valores en columnas
    dataframe['gender'] = dataframe['gender'].map({'Male':      'M', 
                                                   'Female':    'F', 
                                                   'Femal':     'F', 
                                                   'female':    'F', 
                                                   'M':         'M',
                                                   'F':         'F'})

    dataframe['state'] = dataframe['state'].map({'Washington':'Washington',
                                'Arizona':'Arizona',
                                'Nevada':'Nevada',
                                'California':'California',
                                'Oregon':'Oregon',
                                'Cali': 'California',
                                'AZ': 'Arizona',
                                'WA': 'Washington'})

    dataframe['education'] = dataframe['education'].replace('Bachelors','Bachelor')

    #dataframe['customer_lifetime_value'] = dataframe['customer_lifetime_value'].str.replace('%','')

    dataframe['vehicle_class'] = dataframe['vehicle_class'].replace('Sports Car','Luxury')
    dataframe['vehicle_class'] = dataframe['vehicle_class'].replace('Luxury SUV','Luxury')
    dataframe['vehicle_class'] = dataframe['vehicle_class'].replace('Luxury Car','Luxury')

    # Forzar numericos:

    #dataframe['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors = 'coerce')
     
    # dataframe['number_of_open_complaints'] = dataframe['number_of_open_complaints'].str.replace('/00','')
    # dataframe['number_of_open_complaints'] = dataframe['number_of_open_complaints'].str.replace('1/','')
    # dataframe['number_of_open_complaints'] = pd.to_numeric(dataframe['number_of_open_complaints'], errors = 'coerce')

    # Corregir valores NaN:

    dataframe = dataframe.dropna(how='all') # Rows with all characters NaN do not give information, we can discard them.
    dataframe['customer_lifetime_value'] = dataframe['customer_lifetime_value'].fillna(dataframe['customer_lifetime_value'].mean())
    
    # Detectar si hay duplicados:

    if dataframe.duplicated().sum() == 0:
        print('No hay duplicados')
    else:
        print('Duplicated rows need removing')
        dataframe.drop_duplicates(inplace=True)
        print('Duplicated rows have been removed')
    
    return dataframe

df1 = limpito(df)



No hay duplicados


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [30]:
df1.columns


Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employment_status', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')

In [18]:
#1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have 
# a response of "Yes" to the last marketing campaign.


filtered_df = df1[(df1['total_claim_amount'] >= 1000) & (df1['response'] == 'Yes')]

filtered_df

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,10351,FN44127,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,...,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10373,10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2/8/11,Employed,M,...,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury,Medsize,NaN
10487,10487,IX60941,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,...,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10565,10565,QO62792,Oregon,7840.165778,Yes,Extended,College,1/14/11,Employed,M,...,2.0,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,NaN,NaN,NaN


In [21]:
#2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for
# customers who have responded "Yes" to the last marketing campaign. Write your conclusions.
df2 = df1[df1['response'] == "Yes"]
info_df2 = df2.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().round(2)

info_df2

# Female claims are higher than male claims on average, except on personal auto. 
# Males with personal auto got the higher average.

policy_type     gender
Corporate Auto  F         433.74
                M         408.58
Personal Auto   F         452.97
                M         457.01
Special Auto    F         453.28
                M         429.53
Name: total_claim_amount, dtype: float64

In [25]:
# 3. Analyze the total number of customers who have policies in each state, and then filter the results to only 
# include states where there are more than 500 customers.

df3 = df1.groupby('state')['policy_type'].agg(['count'])

df3

# All of the countries have more than 500 customers.

,count
state,
Arizona,1937
California,3552
Nevada,993
Oregon,2909
Washington,888


In [29]:
# 4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

df4 = df1.groupby(['gender','education'])['customer_lifetime_value'].agg(['max','min','median']).round(2)

df4

# High school got the highest values and median for males. 
# For females, bachelors got the max but the median is in high school too.

max      min   median
gender education                                       
F      Bachelor              73225.96  1904.00  5640.51
       College               61850.19  1898.68  5623.61
       Doctor                44856.11  2395.57  5332.46
       High School or Below  55277.45  2144.92  6039.55
       Master                51016.07  2417.78  5729.86
M      Bachelor              67907.27  1898.01  5548.03
       College               61134.68  1918.12  6005.85
       Doctor                32677.34  2267.60  5577.67
       High School or Below  83325.38  1940.98  6286.73
       Master                50568.26  2272.31  5579.10

In [ ]:
#5. The marketing team wants to analyze the number of policies sold by state and month. 
# Present the data in a table where the months are arranged as columns and the states are arranged as rows.


In [39]:
# Asegúrate de que la columna Date esté en formato datetime
df1['effective_to_date'] = pd.to_datetime(df1['effective_to_date'], format='%m/%d/%y')

# Extraer el mes de la columna Date
df1['month'] = df1['effective_to_date'].dt.month

df5 = df1.pivot_table(index='state', columns='month', values=['policy'], aggfunc='count')

df5

policy      
month           1     2
state                  
Arizona      1008   929
California   1918  1634
Nevada        551   442
Oregon       1565  1344
Washington    463   425

In [45]:
# 6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 
# states with the highest number of policies sold.

#*Hint:*
#- *To accomplish this, you will first need to group the data by state and month, then count the number of policies 
# sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
#- *Next, you will select the top 3 states with the highest number of policies sold.*
#- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the 
# top 3 states.*

condition = ['California','Oregon','Arizona']
df6 = df5.loc[condition]
df6

policy      
month           1     2
state                  
California   1918  1634
Oregon       1565  1344
Arizona      1008   929

In [47]:
#7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.
#Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who 
# responded "Yes") by marketing channel.

df7 = df1.groupby(['sales_channel', 'response'])['response'].count()

df7.head()

# Call center always got a No response. All channels got bad response but Agent got the most Yes.



sales_channel  response
Agent          No          3148
               Yes          742
Branch         No          2539
               Yes          326
Call Center    No          1792
Name: response, dtype: int64